In [4]:
import tensorflow as tf
from matplotlib import pyplot as plt

# Set up paths
train_dir = 'final_image_data/train'
val_dir = 'final_image_data/val'

# Load training and validation datasets using image_dataset_from_directory
train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=train_dir,
    labels='inferred',
    label_mode='binary',
    batch_size=10,
    image_size=(512, 512),
    shuffle=True
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=val_dir,
    labels='inferred',
    label_mode='binary',
    batch_size=10,
    image_size=(512, 512)
)

# Normalize pixel values to the range [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))

print("Datasets loaded and normalized successfully.")

Found 4800 files belonging to 2 classes.
Found 1200 files belonging to 2 classes.
Datasets loaded and normalized successfully.


In [5]:
from tensorflow import keras
from keras import layers, models,regularizers,callbacks
# model
model = models.Sequential()
model.add(layers.Conv2D(32, (5,5), activation='relu', input_shape=(512, 512, 3)))
model.add(layers.MaxPooling2D((4, 4), strides=4))
model.add(layers.Conv2D(32, (3, 3), activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.MaxPooling2D((1, 3), strides=(1,3)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='linear',kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(256, activation='relu',kernel_regularizer=regularizers.l2(0.0001)))
model.add(layers.Dropout(0.8))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy',
                       tf.keras.metrics.TrueNegatives(),
                       tf.keras.metrics.TruePositives(),
                       tf.keras.metrics.FalseNegatives(),
                       tf.keras.metrics.FalsePositives()])


d:\Major1\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Training the model with EarlyStopping
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    shuffle=True,
    callbacks=[callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)]
)

# Plotting loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Plotting accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Epoch 1/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 343s 707ms/step - accuracy: 0.5976 - false_negatives: 496.7734 - false_positives: 411.2765 - loss: 3.4282 - true_negatives: 795.4304 - true_positives: 706.4990 - val_accuracy: 0.7683 - val_false_negatives: 67.0000 - val_false_positives: 211.0000 - val_loss: 2.8428 - val_true_negatives: 389.0000 - val_true_positives: 533.0000
Epoch 2/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 331s 690ms/step - accuracy: 0.7751 - false_negatives: 273.3368 - false_positives: 260.4740 - loss: 2.7861 - true_negatives: 946.5738 - true_positives: 929.5946 - val_accuracy: 0.8583 - val_false_negatives: 74.0000 - val_false_positives: 96.0000 - val_loss: 2.3136 - val_true_negatives: 504.0000 - val_true_positives: 526.0000
Epoch 3/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 859s 868ms/step - accuracy: 0.8301 - false_negatives: 221.1143 - false_positives: 186.0125 - loss: 2.3035 - true_negatives: 1020.0457 - true_positives: 982.8066 - val_accuracy: 0.9067 - val_false_negatives: 57.0000 - val_false_

In [ ]:
results = model.evaluate(val_dataset)

# Print the results
print("Loss:", results[0])
print("Accuracy:", results[1])
print("True Negatives:", results[2])
print("True Positives:", results[3])
print("False Negatives:", results[4])
print("False Positives:", results[5])

120/120 ━━━━━━━━━━━━━━━━━━━━ 22s 184ms/step - accuracy: 0.9841 - false_negatives: 0.0000e+00 - false_positives: 10.5455 - loss: 0.5068 - true_negatives: 294.0909 - true_positives: 305.2810
Loss: 0.5052415132522583
Accuracy: 0.98416668176651
True Negatives: 581.0
True Positives: 600.0
False Negatives: 0.0
False Positives: 19.0


In [10]:
model.save("saved_model/my_cnn_model.h5")